##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [3]:
import os
import time
import google.generativeai as genai

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()



In [33]:
system_instruction = """ Jesteś pomocnym edytorem. Czytasz komentarz i przypisujesz mu jeden lub kilka aspektów z listy.
<Aspekty> są zaznaczone a po; jest ich definicja, ale jeśli jako edytor uznasz to za stosowne możesz ją rozszerzyć.

<Zawartość Merytoryczna>;
Zrozumiałość: Czy treść jest łatwa do zrozumienia?
Szczegółowość: Czy treść jest wystarczająco szczegółowa?
Aktualność: Czy treść jest aktualna i zgodna z najnowszą wiedzą?
Praktyczność: Czy treść jest praktyczna i użyteczna w realnych sytuacjach?
Kompletność: Czy wszystkie istotne tematy są omówione?

<Forma Prezentacji>;
Struktura: Czy kurs ma logiczną strukturę i kolejność tematów?
Język: Czy język jest jasny i precyzyjny?
Wizualizacja: Czy materiały wizualne (slajdy, grafiki, filmy) są atrakcyjne i pomocne?
Interaktywność: Czy kurs zawiera elementy interaktywne (quizy, zadania, fora dyskusyjne)?

<Dostępność>;
Czy kurs jest dostępny na różnych urządzeniach i dla osób z różnymi potrzebami?

<Jakość Techniczna>;
Audio: Czy jakość dźwięku jest dobra?
Wideo: Czy jakość wideo jest dobra?
Nawigacja: Czy nawigacja po kursie jest łatwa i intuicyjna?

<Dostępność Materiałów>;
Czy materiały dodatkowe (PDFy, ćwiczenia) są łatwo dostępne i działają poprawnie?

<Stabilność Platformy>;
Czy platforma działa stabilnie i bez problemów technicznych?

<Lektor/Prowadzący>;
Kompetencje: Czy prowadzący posiada odpowiednią wiedzę i umiejętności?
Przejrzystość Wykładu: Czy prowadzący przekazuje wiedzę w sposób jasny i zrozumiały?
Zaangażowanie: Czy prowadzący jest zaangażowany i motywuje do nauki?
Język: Czy lektor mówi językiem, który jest dla Ciebie naturalny (w przypadku dubbingu lub AI, jakość głosu)?

<Testy/Zadania>;
Zgodność: Czy pytania testowe są zgodne z treścią kursu?
Poziom Trudności: Czy testy są odpowiednio trudne?
Informacja Zwrotna: Czy uczestnicy otrzymują informację zwrotną na temat swoich wyników?
Praktyczność: Czy testy sprawdzają umiejętność zastosowania wiedzy w praktyce?

<Język i Tłumaczenie>;
Poprawność Tłumaczenia: Czy tłumaczenie jest poprawne i zrozumiałe?
Dostosowanie Kulturowe: Czy treść jest dostosowana do kultury i kontekstu językowego?
Spójność Językowa: Czy tłumaczenie jest spójne i używa tej samej terminologii w całym kursie?

<Certyfikat>;
Dostępność: Czy łatwo uzyskać certyfikat po ukończeniu kursu?
Wartość: Czy certyfikat jest ceniony przez pracodawców?

<Długość Kursu>;
Czy długość kursu jest odpowiednia?
Organizacja -  Czy kurs jest dobrze zorganizowany?

<Motywacja>;
Czy kurs motywuje do dalszej nauki?

<Reklama>; Skuteczność i trafność reklamy kursu.

<Pozostałe>: wybierz ten aspekt jeśli w komentarzy jest coś co nie pasuje do rzadnego ze zdefiniowanych aspektów

Do każdego aspektu jaki przypisujesz oszacuj zabarwienie opinii, czy kest to pochwała czy krytyka danego aspekty.
Pochwały zazacz (+), krytykę (-) a opinie neutralne ().
Przeanalizuj każdy komentarz i przypisz jeden lub kilka aspektów. **Zwróć wynik analizy w formacie JSON.** , na przykład:
 [
       {
         "komentarz": "Szkolenie było świetne!",
         "aspekty": ["Zawartość Merytoryczna", "Lektor/Prowadzący"],
         "zabarwienie": ["+", "+"]
       },
       {
         "komentarz": "...",
         "aspekty": [...],
         "zabarwienie": [...]
       }
     ]
"""

In [34]:
# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 100000,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash",
  generation_config=generation_config,
  system_instruction=system_instruction)



In [ ]:
# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("comments_sample.txt", mime_type="text/plain"),
]

# Some files have a processing delay. Wait for them to be ready.
wait_for_files_active(files)

## Call `generate_content`

In [25]:
contents = "Jak się masz?"

In [35]:
contents  = { "role": "user",
      "parts": [
        files[0],
        "Przeanalizuj każdy komentarz i przypisz jeden lub kilka aspektów. Zapisz wynik analizy w formacie JSON "]
}


In [36]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = model

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)


In [50]:
# Assuming response.text is a long string
start_index = 8  # Start position of the desired part
end_index = 300  # End position of the desired part

# Extract and print the substring
print(response.text[start_index:end_index])

[
  {
    "komentarz": "Polecam tym, którzy chcą posiąść podstawową wiedzę o Agile.",
    "aspekty": [
      "Zawartość Merytoryczna"
    ],
    "zabarwienie": [
      "+"
    ]
  },
  {
    "komentarz": "Filmy instruktażowe powinny być nagranie z udziałem polskiego lektora. Nie zawsze jest 


In [52]:
 data = json.loads(response.text[start_index:])

JSONDecodeError: Unterminated string starting at: line 1071 column 18 (char 22007)

In [48]:
import json

try:
    data = json.loads(response.text[start_index:end_index])  # Attempt to parse as JSON

    # Limit to the first 3 examples (or fewer if there are less than 3)
    for item in data[:3]:
        print(f"Komentarz: {item['komentarz']}")
        print(f"Aspekty: {item['aspekty']}")
        print(f"Zabarwienie: {item['zabarwienie']}")
        print("-" * 20)  # Separator

except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    print(f"Generated content: {response.text}")  # Print for debugging

Error parsing JSON: Unterminated string starting at: line 3 column 5 (char 10)
Generated content: ```json
[
  {
    "komentarz": "Polecam tym, którzy chcą posiąść podstawową wiedzę o Agile.",
    "aspekty": [
      "Zawartość Merytoryczna"
    ],
    "zabarwienie": [
      "+"
    ]
  },
  {
    "komentarz": "Filmy instruktażowe powinny być nagranie z udziałem polskiego lektora. Nie zawsze jest możliwość czytania napisów.",
    "aspekty": [
      "Lektor/Prowadzący",
      "Dostępność"
    ],
    "zabarwienie": [
      "-",
      "-"
    ]
  },
  {
    "komentarz": "Bardzo ciekawe szkolenie, dużo zróżnicowanych materiałów. Nic bum nie zmieniał.",
    "aspekty": [
      "Zawartość Merytoryczna"
    ],
    "zabarwienie": [
      "+"
    ]
  },
  {
    "komentarz": "Możecie dodać więcej praktycznych przykładów",
    "aspekty": [
      "Zawartość Merytoryczna"
    ],
    "zabarwienie": [
      "-"
    ]
  },
  {
    "komentarz": "Nie wiem, kurs ok",
    "aspekty": [
      "Pozostałe"
    ]

In [42]:
# Assuming response.text is a long string
start_index = 100  # Start position of the desired part
end_index = 200    # End position of the desired part

# Extract and print the substring
print(response.text[start_index:end_index])

spekty": [
      "Zawartość Merytoryczna"
    ],
    "zabarwienie": [
      "+"
    ]
  },
  {
    "


google.ai.generativelanguage_v1beta.types.content.Content

In [20]:
 # If there are candidates, get the content of the first one.
if response.candidates:
    generated_content = response.candidates[0].content

    # Attempt to parse the generated content as JSON.
    try:
        data = json.loads(generated_content)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Generated content: {generated_content}") # Print the content to help with debugging
    else:
        # Save the JSON data to a file
        with open('output.json', 'w') as f:
            json.dump(data, f, indent=4)  # indent for better readability
else:
    print("No candidates found in the response.")

TypeError: the JSON object must be str, bytes or bytearray, not Content